In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

logic:
1. at the last second block in current epoch, get the effective stakes in current epoch (effective stakes donot change during the whole epoch, and we cannot get the effective stake info at the last block, so we choose the last second)
2. at the last block in current epoch, get the validator reward (accumulated reward in current block - acculated reward in last block)
3. at the first block in new epoch, get the new effective stakes in new epoch, check the effective stakes changes. get the block reward, check the difference of block reward.
4. check if the effective stake increases, whether the validator reward increases, if the effective stake decreases, whether the validator reward decreases.

Note: at the last block in every epoch, in the beginning, the epos-status will change to eligible / not eligible
the currently in committee will be false
and there is no metrics

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [6]:
def getStakeAndRewards():
    stake = dict()
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            effective_stake = float(i['metrics']['total-effective-stake'])
            stake[address] = effective_stake
            reward[address] = reward_accumulated
    return stake, reward

In [7]:
def getRewards():
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        reward_accumulated = i['lifetime']['reward-accumulated']
        reward[address] = reward_accumulated
    return reward

In [8]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            if v - map1[k] != 0:
                map3[k] = v - map1[k]
    return map3

In [9]:
def getEpoch():
    method = 'hmy_getEpoch'
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [10]:
def R11_test():
    print("Test-R11: Earning is proportional to effective stake ")
    num = int(input("enter the number of epoches you want to test: "))
    
    iterations = 0
    while iterations < num:
        print("test " + str(iterations+1) + " will begin ...")
        block = getBlockNumber()
        last_block = getLastBlockOfCurrentEpoch()
        if block == last_block:
            new_block = block+1
            while block < new_block:
                block = getBlockNumber()
        epoch = getEpoch()
        print("current epoch: ", epoch)
        print("current and last block numbers", block, last_block)
        second_last_block = last_block - 1
        while block < second_last_block:
            block = getBlockNumber()
        print("second last block in current epoch reached", block, "will begin testing...")
        stakes, rewards = getStakeAndRewards()
        
        # in the last block, we can not get the total effective stakes, no metrics. 
        new_block = block + 1
        while block < new_block:
            block = getBlockNumber()
        print("last block in current epoch reached", block, "will calculate the reward per block")
        new_rewards = getRewards()
        reward_per_block = diffAndFilter(rewards, new_rewards)

        new_block = block + 1
        while block < new_block:
            block = getBlockNumber()
        print("new epoch reached")
        last_block = getLastBlockOfCurrentEpoch()
        print("current and last block numbers", block, last_block)
        
        second_last_block = last_block - 1
        while block < second_last_block:
            block = getBlockNumber()
        print("second last block in new epoch reached", block, "will begin testing...")
        new_stakes, rewards = getStakeAndRewards()

        new_block = block + 1
        while block < new_block:
            block = getBlockNumber()
        print("last block in new epoch reached", block, "will calculate the reward per block")
        new_rewards = getRewards()
        reward_per_block_next = diffAndFilter(rewards, new_rewards)
        
        reward_diff = diffAndFilter(reward_per_block, reward_per_block_next)
        # get the validators whose effective stake changes
        stake_diff = diffAndFilter(stakes, new_stakes)

        if not stake_diff:
            print("in this iteration, no validators change the effective stake")
        if not reward_diff:
            print("in this iteration, no validators change the reward")
            
        flag = True
        for k,v in stake_diff.items():
            if k in reward_diff:
                if v > 0: 
                    if reward_diff[k] <= 0:
                        flag = False
                        print("Test-R11: Fail")
                        print(k, "effective stake increase: ", v)
                        print("earning doesn't increase, earning changes: ", reward_diff[k])
                if v < 0:
                    if reward_diff[k] >= 0:
                        flag = False
                        print("Test-R11: Fail")
                        print(k, "effective stake decrease: ", v)
                        print("earning doesn't decrease, earning changes: ", reward_diff[k])
                        
        if flag:
            print("Test-R11: Success")
        iterations += 1        

In [11]:
R11_test()

Test-R11: Earning is proportional to effective stake 
enter the number of epoches you want to test: 2
test 1 will begin ...
current epoch:  368
current and last block numbers 14009 14021
second last block in current epoch reached 14020 will begin testing...
last block in current epoch reached 14021 will calculate the reward per block
new epoch reached
current and last block numbers 14022 14059
second last block in new epoch reached 14058 will begin testing...
last block in new epoch reached 14059 will calculate the reward per block
Test-R11: Fail
one1vzsj3julf0ljcj3hhxuqpu6zvadu488zfrtttz effective stake decrease:  -3.399999999999998e+21
earning doesn't decrease, earning changes:  25340786769901400
Test-R11: Fail
one1s7fp0jrmd97estwye3mhkp7xsqf42vn5x2sfqy effective stake decrease:  -1.8399999999999941e+22
earning doesn't decrease, earning changes:  137138375460642864
Test-R11: Fail
one1km7xg8e3xjys7azp9f4xp8hkw79vm2h3f2lade effective stake decrease:  -4.5999999999999853e+21
earning doe